In [ ]:
# !conda install -y -c anaconda psycopg2
# !conda install -y -c anaconda requests
# !conda install -c anaconda pytz
# from requests import Session
# from lxml import html
# from datetime import datetime
# import time
# import random
# import sys
# import logging
# import psycopg2

In [46]:
from requests import Session
from lxml import html
from datetime import datetime
import time
import random
import sys
import logging
import psycopg2
from psycopg2 import sql


#rds settings
rds_host  = "database-spotifier.c5eevkz7wazj.us-east-2.rds.amazonaws.com"
name = "tally_ds" 
db_name = "postgres" 
password = "P@ssw0rd"


logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = psycopg2.connect(host=rds_host, user=name, password=password, database=db_name, port=5432, connect_timeout=5)
except psycopg2.OperationalError as e:
    logger.error("ERROR: Unexpected error: Could not connect to postGreSQL instance.")
    logger.error(e)
    sys.exit()

logger.info("SUCCESS: Connection to RDS postGreSQL instance succeeded")


def yelpScraper(business_id):
    base_url = "https://www.yelp.com/biz/" # add business id
    api_url = "/review_feed?sort_by=date_desc&start=" # add number
    results = []
    for n in range(1):
        with Session() as s:
            url = base_url + business_id + api_url + str(n*20)    
            with s.get(url, timeout=5) as r:    
                if r.status_code==200:
                    response = dict(r.json()) 
                    _html = html.fromstring(response['review_list']) 
                    dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                    dates = [datetime.strptime(d.strip(), format("%m/%d/%Y")) for d in dates]
                    stars = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")
                    stars = [float(s.split(' ')[0]) for s in stars]
                    texts = [e.text for e in _html.xpath("//div[@class='review-content']/p")]
                    results = results + [[date, star, text] for date, star, text in zip(dates, stars, texts)]
        time.sleep(random.uniform(0.1, 0.5))
    return results, business_id
def dbConnect(results, business_id):
    item_count = 0
    with conn.cursor() as cur:
        for i in results:
            x = i[0]
            x = x.strftime("%Y-%m-%d")
            y = float(i[1])
            z = i[2]
#             cur.execute("INSERT INTO lab.yelp_scraping (business_id, stars, text) VALUES (%s, %s, %s);", (business_id, y, z))
            cur.execute(sql.SQL("insert into {} values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
                        .format(sql.Identifier('lab','yelp_scraping')),
                        ['uuid-ossp',#uuid
                         'NULL',# review id
                         business_id,
                         'NULL', # user id
                         y, # stars
                         datetime.now(), # datetime
                         x,# date of review
                         current_timestamp,# time without timezone,
                         z,# review text
                         now(),# time with timezone
                         ])

            conn.commit()
            cur.execute("select * from lab.yelp_scraping")
            for row in cur:
                item_count += 1
                logger.info(row)
        conn.commit()
    return "Added %d items from RDS postGreSQL table" %(item_count)
    
# def lambda_handler(event, context):
#     yelpScraper("jga_2HO_j4I7tSYf5cCEnQ")
#     dbConnect()
#     return "it worked"

business_id = "jga_2HO_j4I7tSYf5cCEnQ"
results = yelpScraper(business_id)
dbConnect(results, business_id)

AttributeError: 'list' object has no attribute 'strftime'

In [5]:
from requests import Session
from lxml import html
from datetime import datetime
import time
import random
import sys
import logging
import psycopg2
from psycopg2 import sql
import pytz


#rds settings
rds_host  = "database-spotifier.c5eevkz7wazj.us-east-2.rds.amazonaws.com"
name = "tally_ds" 
db_name = "postgres" 
password = "P@ssw0rd"


logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = psycopg2.connect(host=rds_host, user=name, password=password, database=db_name, port=5432, connect_timeout=5)
except psycopg2.OperationalError as e:
    logger.error("ERROR: Unexpected error: Could not connect to postGreSQL instance.")
    logger.error(e)
    sys.exit()

logger.info("SUCCESS: Connection to RDS postGreSQL instance succeeded")


def yelpScraper(business_id):
    base_url = "https://www.yelp.com/biz/" # add business id
    api_url = "/review_feed?sort_by=date_desc&start=" # add number
    results = []
    for n in range(1):
        with Session() as s:
            url = base_url + business_id + api_url + str(n*20)    
            with s.get(url, timeout=5) as r:    
                if r.status_code==200:
                    response = dict(r.json()) 
                    _html = html.fromstring(response['review_list']) 
                    dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                    dates = [datetime.strptime(d.strip(), format("%m/%d/%Y")) for d in dates]
                    stars = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")
                    stars = [float(s.split(' ')[0]) for s in stars]
                    texts = [e.text for e in _html.xpath("//div[@class='review-content']/p")]
                    results = results + [[date, star, text] for date, star, text in zip(dates, stars, texts)]
        time.sleep(random.uniform(0.1, 0.5))
    return results
def dbConnect(results, business_id):
    item_count = 0
    with conn.cursor() as cur:
        for i in results:
            x = i[0]
            x = x.strftime("%Y-%m-%d")
            y = float(i[1])
            z = i[2]
#             cur.execute("INSERT INTO lab.yelp_scraping (business_id, stars, text) VALUES (%s, %s, %s);", (business_id, y, z))
            cur.execute(sql.SQL("insert into {} (review_id, business_id, user_id, stars, datetime, date, time, text, timestamp) values ( %s, %s, %s, %s, %s, %s, %s, %s, %s);")
                        .format(sql.Identifier('lab','yelp_scraping')),
                        [#'uuid-ossp',#uuid
                         'NULL',# review id
                         business_id,
                         'NULL', # user id
                         y, # stars
                         datetime.now(), # datetime
                         x,# date of review
                         datetime.now(),# time without timezone,
                         z,# review text
                         datetime.now(pytz.utc),# time with timezone
                         ])

            conn.commit()
            cur.execute("select * from lab.yelp_scraping")
            for row in cur:
                item_count += 1
                logger.info(row)
        conn.commit()
    return "Added %d items from RDS postGreSQL table" %(item_count)
    
# def lambda_handler(event, context):
#     yelpScraper("jga_2HO_j4I7tSYf5cCEnQ")
#     dbConnect()
#     return "it worked"

business_id = "9m14fTWyiBi4AHN32rOrDg"
results = yelpScraper(business_id)
dbConnect(results, business_id)

'Added 210 items from RDS postGreSQL table'